# Rasterizing a voronoi diagram

In [74]:
from PIL import Image, ImageDraw
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np

In [75]:
def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)

def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

Drawing a grid of pixels

In [76]:
res = 100
image_resolution = 1080
offset = 0.5
scale_factor = image_resolution / res

img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)

for x in range(res):
    for y in range(res):
        center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, 'blue')
# draw.rectangle([0,0,image_resolution,image_resolution],fill="white")
img.save('pixels.png')

Rasterizing a single polygon

In [77]:
triangle = np.array([[15,10],[75,20],[60,60]])

# for vert in triangle:
#     x = vert[0] * scale_factor
#     y = vert[1] * scale_factor
#     center_ellipse(x,y,10,'white')

# img.save('pixels.png')

Edge function

In [78]:
def Edge_Function(_v0, _v1, _p):
    #Assumes clockwise order
    result = (_p[0] - _v0[0]) * (_v1[1] - _v0[1]) - (_p[1] - _v0[1]) * (_v1[0] - _v0[0])
    
    if result <= 0:
        return True
    else:
        return False

Single edge case

In [79]:
img = Image.open('pixels.png')
draw = ImageDraw.Draw(img)

for x in range(res):
    for y in range(res):
        state = Edge_Function(triangle[0],triangle[1],[x,y])
        if state:
            center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, 'blue')
        else:
            center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, 'red')
img.save('pixels.png')

Rasterizing a triangle

In [80]:
img = Image.open('pixels.png')
draw = ImageDraw.Draw(img)
op_count = 0
for x in range(res):
    for y in range(res):
        edge0 = Edge_Function(triangle[0],triangle[1],[x,y])
        edge1 = Edge_Function(triangle[1],triangle[2],[x,y])
        edge2 = Edge_Function(triangle[2],triangle[0],[x,y])
        
        
        if edge0 and edge1 and edge2:
            center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, 'red')
        else:
            center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, 'blue')
        op_count += 1
img.save('pixels.png')
print(f"total operations: {op_count}")

total operations: 10000


Optimizing by reducing pixel search to bounding box

In [81]:
def Bounding_Box(polygon):
    x_vals = []
    y_vals = []
    for vert in polygon:
        x_vals.append(vert[0])
        y_vals.append(vert[1])

    x_min = round(np.min(x_vals))
    x_max = round(np.max(x_vals)) + 1 #account for
    y_min = round(np.min(y_vals))
    y_max = round(np.max(y_vals)) + 1 #account for

    return ([x_min, x_max],[y_min,y_max])

img = Image.open('pixels.png')
draw = ImageDraw.Draw(img)

bbox = Bounding_Box(triangle)
op_count = 0
print(bbox)
for x in range(*bbox[0]):
    for y in range(*bbox[1]):
            
            edge0 = Edge_Function(triangle[0],triangle[1],[x,y])
            edge1 = Edge_Function(triangle[1],triangle[2],[x,y])
            edge2 = Edge_Function(triangle[2],triangle[0],[x,y])
            
            
            if edge0 and edge1 and edge2:
                center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, 'red')
            else:
                center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, 'blue')
                center_ellipse((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, 2, 'white')
            op_count += 1
img.save('pixels.png')
print(f"total operations: {op_count}")


([15, 76], [10, 61])
total operations: 3111


Rasterizing a Polygon

Must be convex & ordered

In [82]:
polygon = np.array([[15,10],[75,20],[80,40],[70,80],[50,90],[10,50]])

img = Image.open('pixels.png')
draw = ImageDraw.Draw(img)


print(bbox)

def Rasterize_Polygon(polygon, color, path):

    
    bbox = Bounding_Box(polygon)
    print(bbox)
    for x in range(*bbox[0]):
        for y in range(*bbox[1]):
                state = True
                vertex_count = len(polygon)

                for i in range(vertex_count):
                    v0 = i
                    v1 = i + 1
                    if v1 == vertex_count:
                        v1 = 0
                    edge_state = Edge_Function(polygon[v0],polygon[v1],[x,y])
                    state = state and edge_state
                
                # if edge0 and edge1 and edge2 and edge3 and edge4 and edge5:
                if state:
                    center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, color)
                # else:
                #     center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, background)
                #     center_ellipse((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, 2, 'white')



    # img.save(path)

for vert in polygon:
    x = vert[0] * scale_factor
    y = vert[1] * scale_factor
    center_ellipse(x,y,10,'white')

Rasterize_Polygon(polygon,'red','pixels.png')

([15, 76], [10, 61])
([10, 81], [10, 91])


Sorting a polygon:

In [83]:
unsorted_polygon = np.array([[75,20],[80,40],[15,10],[70,80],[50,90],[10,50]])

img = Image.open('pixels.png')
draw = ImageDraw.Draw(img)

r = 5
for vert in unsorted_polygon:
    x = vert[0] * scale_factor
    y = vert[1] * scale_factor
    center_ellipse(x,y,r,'white')
    r *= 1.5
# img.save('pixels.png')


In [99]:
def Calculate_Centroid(polygon):
    x = 0
    y = 0
    for vert in polygon:
        x += vert[0]
        y += vert[1]
    x /= len(polygon)
    y /= len(polygon)

    return [x,y]

def Sort_Vertices(polygon):
    angles = []
    centroid = Calculate_Centroid(polygon)
    # vA = x vector starting at centroid
    vA = np.array([centroid[0] + 300,centroid[1],0])
    s = 5
    for vert in polygon:
        x = vert[0]
        y = vert[1]
        #vB = vector between centroid and vertex
        vB = np.array([x - centroid[0], y - centroid[1],0])
        
        numerator = np.dot(vA,vB)
        denominator = np.linalg.norm(vA) * np.linalg.norm(vB)
        theta = np.arccos(numerator / denominator)
        
        # if y < centroid[1]:
        #     theta = -theta

        sign = np.cross(vA,vB) / np.linalg.norm(np.cross(vA,vB))
        theta *= sign[2]

        angles.append(theta)
        width = round(2 * theta * np.pi)
        print(f"theta: {theta},sign: {sign[2]}")

        # draw.line((centroid[0],centroid[1],x,y),fill='white', width=width) #Drawing vB
        center_ellipse(x,y,s,'green')
        s += 2

    draw.line((centroid[0],centroid[1],vA[0],vA[1]),fill='red', width=width) #Drawing vA


    sorted_verts = np.array([x for _, x in sorted(zip(angles, polygon))])
    start_index = sorted(angles).index(min(angles))
    return sorted_verts, start_index


def Outline_Poly(polygon,color,width):
    c = 50
    for i, point in enumerate(polygon):

        k = i + 1
        if k == len(polygon):
            k = 0
        point2 = polygon[k]
        # point = [p * scale_factor for p in point]
        # point2 = [p * scale_factor for p in point2]

        draw.line((point[0],point[1],point2[0],point2[1]),fill=f'rgb({c},{c},{c})', width=width)
        c += 25

In [100]:
res = 100
image_resolution = 1080
offset = 0.5
scale_factor = image_resolution / res

polygon = np.array([[10,50],[15,10],[80,40],[75,20],[70,80],[50,90]])

polygon = np.array([
 [195.40786101, 265.28646685],
 [185.41111442, 294.97039045],
 [364.14450089, 174.85409131],
 [470.34674382, 279.41080524],
 [388.01363268, 439.38456405],
 [290.11147292, 454.72079006]])
 

img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)

# sorted_polygon = Sort_Vertices([p*scale_factor for p in polygon])
sorted_polygon, min_index = Sort_Vertices(polygon)
[print(v,s) for v,s in zip(polygon,sorted_polygon)]
print(f"min index: {min_index}")
Outline_Poly(sorted_polygon,'rgb(255,255,255)', 6)

# for p in sorted_polygon:
#     center_ellipse(p[0],p[1],10,'white')
centroid = Calculate_Centroid(polygon)

center_ellipse(centroid[0] * scale_factor,centroid[1] * scale_factor,10,'white')

img.save('pixels3.png')

theta: 3.0787529561375653,sign: 1.0
theta: 2.840522818872819,sign: 1.0
theta: -1.7208586643268877,sign: -1.0
theta: -0.7219467976939639,sign: -1.0
theta: 0.5553942763071783,sign: 1.0
theta: 1.2780846424593413,sign: 1.0
[195.40786101 265.28646685] [364.14450089 174.85409131]
[185.41111442 294.97039045] [470.34674382 279.41080524]
[364.14450089 174.85409131] [388.01363268 439.38456405]
[470.34674382 279.41080524] [290.11147292 454.72079006]
[388.01363268 439.38456405] [185.41111442 294.97039045]
[290.11147292 454.72079006] [195.40786101 265.28646685]
min index: 0
